In [1]:
import psycopg2 as pg
import sqlalchemy as db
import pandas as pd
import lyricsgenius as genius
import numpy as np
import pickle
from sqlalchemy.orm import sessionmaker
import time

# In this notebook I grab all of the songs and their lyrics

### Lets start with fake user creds for genius, to get your own sign up [here](https://genius.com/api-clients)

In [1]:
geniusCreds = "fake"

# Connect to Database

In [3]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  
    'dbname': 'lyricsdb',  
}

connection = pg.connect(**connection_args)

# grab the song lyrics

In [15]:
"""
song_df = pd.DataFrame()
song_df["artist_id"] = np.nan
song_df["song_title"] = np.nan
song_df["lyrics"] = np.nan

with open('../artist_data/song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)
""";

#### first start by loading in what we have so far.This is a continuous process so everytime we restart we want to pick up from here e left off

In [7]:
with open('../artist_data/artist_dataframe.pkl', 'rb') as f:
    artist_df = pickle.load(f)
    
with open('../artist_data/song_df.pkl', 'rb') as f:
    song_df = pickle.load(f)

In [31]:
query = 'SELECT * FROM songs;'
song_df = pd.read_sql(query, connection)

In [32]:
query = 'SELECT * FROM artist_info;'
artist_df = pd.read_sql(query, connection)

In [33]:
song_df

,artist_id,song_title,lyrics
0,1083,Radio 2,
1,1274,Sound Round,I'm young in my camper van\nThe world feels ol...
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
...,...,...,...
56304,1303,Heavy Music (Part 2),Ah come on with me baby\nYou're gonna have a g...
56305,1303,Real Mean Bottle,No man ever sounded so lonesome\nNo man ever m...
56306,1303,East Side Story (Vocal),Beneath the bare light bulb above\nShe gazed i...
56307,1303,Very Few,Very few in life composed a symphony\nVery few...


In [34]:
artist_df.head()

,artist_id,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
0,0,Tupac,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/115/228...,None,0
1,1,The Notorious B.I.G.,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/111/221...,None,1
2,2,Eminem,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/47/9378...,None,2
3,3,Kendrick Lamar,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/3107/62...,None,3
4,4,Dr. Dre,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/45/8916...,None,4


### Heres a list of artists whose names were inconsistent between ranker and genius

In [35]:
missing_artists = ["Bob Seger & the Silver Bullet Band", "Simon and Garfunkel", "Santana", 
                   "Joan Jett and the Blackhearts", "Smokey Robinson &amp; the Miracles"]
replacing_names = ["Bob Seger", "Simon & Garfunkel", "Santana", "Joan Jett & the Blackhearts", 
                   "Smokey Robinson & the Miracles"]


no_artists = ["The Carpenters"]

In [36]:
# grab just the rock artists that havnt been grabbed yet
rock_artists = artist_df[artist_df["genre"].str.contains('Rock', regex=False) 
                         & artist_df["artist_picture_genius_url"].isnull() 
                        & ~artist_df["artist_name"].isin(no_artists)
                        ]

In [37]:
rock_artists

,artist_id,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
1484,1329,Joan Jett & the Blackhearts,Classic Rock,https://imgix.ranker.com/user_node_img/3379/67...,None,75


In [38]:
# connect to genius
api = genius.Genius(geniusCreds, verbose = False, remove_section_headers = True, skip_non_songs = True, excluded_terms = ["(Remix)", "(Live)"])

In [39]:
# make connecting to specific tables in the database
engine = db.create_engine('postgresql://localhost:5432/lyricsdb')
connection = engine.connect()
metadata = db.MetaData()
artist_table = db.Table('artist_info', metadata, autoload=True, autoload_with=engine)
song_table = db.Table('songs', metadata, autoload=True, autoload_with=engine)

In [40]:
def grab_songs(i, artist, song_df, artist_df, connection):
    """
    This function takes in and artist sting, a song and artists df as well as a db connection
    IT grabs all the song lyrics from the artist and updates the dataframe and data tables
    """
    print("Grabbing songs from {}".format(artist["artist_name"]))
    genius_artist = api.search_artist(artist["artist_name"]);
    genius_img = genius_artist.image_url
    artist_id = artist["artist_id"]
    print("Looping Songs")
    count=1
    sql_song_insert_list = []
    for s in genius_artist.songs:
        song_title = s.title
        lyrics = s.lyrics

        # add to df
        song_df = song_df.append({'artist_id' : int(artist_id) , 'song_title' : song_title, 'lyrics':lyrics} , ignore_index=True)
        # add to db
        # insert = song_table.insert().values(artist_id=int(artist_id), song_title=song_title, lyrics=lyrics)
        # connection.execute(insert)

        sql_insert_dict = {'artist_id': int(artist_id), 'song_title' : song_title, 'lyrics': lyrics}
        sql_song_insert_list.append(sql_insert_dict)

        print("appended song {}, {}".format(count, song_title))
        count+=1



    # add value to Artist Dataframe
    artist_df.loc[artist_df["artist_name"] == artist["artist_name"], "artist_picture_genius_url"] = genius_img

    # add value to db
    update = artist_table.update().where(artist_table.c.artist_name==artist["artist_name"]).values(artist_picture_genius_url=genius_img)
    connection.execute(update)

    # insert all songs to db
    connection.execute(song_table.insert(), sql_song_insert_list)

    with open('../artist_data/artist_dataframe.pkl', 'wb') as f:
        pickle.dump(artist_df, f)

    with open('../artist_data/song_df.pkl', 'wb') as f:
        pickle.dump(song_df, f)

In [41]:
# loop tjhrough the artists in the dataframe to grab their lyrics
for i, artist in rock_artists.iterrows():
    try:
        grab_songs(i, artist, song_df, artist_df, connection)
    except (TypeError, AttributeError):
        print("Type Error, going to sleep then trying again")
        time.sleep(5)
        grab_songs(i, artist, song_df, artist_df, connection)

Grabbing songs from Joan Jett & the Blackhearts
Looping Songs
appended song 1, I Love Rock ’N Roll
appended song 2, I Hate Myself for Loving You
appended song 3, Crimson & Clover
appended song 4, Fake Friends
appended song 5, Do You Wanna Touch Me (Oh Yeah)
appended song 6, Love Is All Around
appended song 7, Different
appended song 8, Fetish
appended song 9, I Want You
appended song 10, Cherry Bomb
appended song 11, The French Song
appended song 12, Love Is Pain
appended song 13, Androgynous
appended song 14, A.C.D.C.
appended song 15, (I’m Gonna) Run Away
appended song 16, Everybody Needs a Hero
appended song 17, Victim Of Circumstance
appended song 18, I Wanna Be Your Dog
appended song 19, I Need Someone
appended song 20, Be Straight
appended song 21, You Drive Me Wild
appended song 22, Soulmates To Strangers
appended song 23, Little Liar
appended song 24, Baby Blue
appended song 25, Kiss On The Lips
appended song 26, TMI
appended song 27, Activity Grrrl
appended song 28, Real Wild 

# All Done!